In [ ]:
!pip install txtai

In [ ]:
! pip install transformers

In [1]:
import glob
import re
#import pandas as pd
# import spacy
import txtai
#from IPython.display import HTML, display

from txtai.embeddings import Embeddings
from txtai.pipeline import Extractor

#pd.set_option("display.max_colwidth", None)
#import cv2
#import matplotlib.pyplot as plt
#import numpy as np

/home/jdownes/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM, GenerationConfig

In [3]:
with open("sample.txt") as file:
    data = file.read().replace('\n',' ')
    # stupid sentence split
    data = data.split(". ")
     
print(data)

['Reston, VA, Thursday, Nov', "2, 2023, — MetroStar's Chief Executive Officer and Co-Founder, Ali Reza Manouchehri, was named a winner of the Greater Washington Government Contractor Awards unveiled on Nov", '1 at the 21st Annual Awards Show at the Ritz-Carlton Tysons Corner', ' Manouchehri earned the award of Executive of the Year in the $75 Million – $300 Million category on Wednesday night', 'Other notable winners included Carissa Christensen of BryceTech, named Executive of the Year in the up to $75 million category, and Carey Smith of Parsons, recognized as Executive of the Year in the over $300 million category', ' Presented by the Northern Virginia Chamber of Commerce and the Professional Services Council, the winners were announced at the evening gala, known as the “Academy Awards of Government Contracting.” In attendance, Manouchehri was humbled and honored to be presented with this award, acknowledging the exceptional talents of his fellow contenders — Sunny Singh from Aeyon,

In [ ]:
'''
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large", model_max_length = 1000)
x = tokenizer(data)
print(x)
'''

In [4]:
#https://github.com/MetroStar/maiden-ai/blob/main/QA-LLM.ipynb

embeddings = Embeddings(
    {"path": "sentence-transformers/multi-qa-mpnet-base-dot-v1", "content": True, "tokenize": True}
)
# Build embeddings index
#https://neuml.github.io/txtai/embeddings/
#simulate user based tags

'''
embeddings.index([{"text": "text to index", "flag": True,
                   "actiondate": "2022-01-01"}])
'''
num_users = 3
user_data=[]
row_id = 0
user_id = 0
for d in data:
    
    #user_data.append((row_id, d, {"user_id": user_id}))
    user_data.append({"text": d, "user_id":user_id})
    user_id +=1
    row_id +=1
    if user_id == num_users:
        user_id = 0 
print(user_data)
embeddings.index(user_data)

[{'text': 'Reston, VA, Thursday, Nov', 'user_id': 0}, {'text': "2, 2023, — MetroStar's Chief Executive Officer and Co-Founder, Ali Reza Manouchehri, was named a winner of the Greater Washington Government Contractor Awards unveiled on Nov", 'user_id': 1}, {'text': '1 at the 21st Annual Awards Show at the Ritz-Carlton Tysons Corner', 'user_id': 2}, {'text': ' Manouchehri earned the award of Executive of the Year in the $75 Million – $300 Million category on Wednesday night', 'user_id': 0}, {'text': 'Other notable winners included Carissa Christensen of BryceTech, named Executive of the Year in the up to $75 million category, and Carey Smith of Parsons, recognized as Executive of the Year in the over $300 million category', 'user_id': 1}, {'text': ' Presented by the Northern Virginia Chamber of Commerce and the Professional Services Council, the winners were announced at the evening gala, known as the “Academy Awards of Government Contracting.” In attendance, Manouchehri was humbled and 

In [5]:
results = embeddings.search("who won the executive of the year award")
print(results)

[{'id': '3', 'text': ' Manouchehri earned the award of Executive of the Year in the $75 Million – $300 Million category on Wednesday night', 'score': 0.7093393802642822}, {'id': '4', 'text': 'Other notable winners included Carissa Christensen of BryceTech, named Executive of the Year in the up to $75 million category, and Carey Smith of Parsons, recognized as Executive of the Year in the over $300 million category', 'score': 0.6936542987823486}, {'id': '6', 'text': '  The MetroStar team celebrating the Executive of the Year award', 'score': 0.640782356262207}]


In [6]:
#https://neuml.github.io/txtai/embeddings/query/

#query = "select object from txtai where similar(:x) AND user_id=(:y)"
#results = embeddings.search(query, parameters={"x": "who won the executive of the year award", "y":1})
query = "select * from txtai where similar('who won the executive of the year award') AND user_id=1"
results = embeddings.search(query)
print(results)

[{'indexid': 4, 'id': '4', 'text': 'Other notable winners included Carissa Christensen of BryceTech, named Executive of the Year in the up to $75 million category, and Carey Smith of Parsons, recognized as Executive of the Year in the over $300 million category', 'tags': None, 'entry': '2023-11-14 10:49:01.846360', 'data': '{"text": "Other notable winners included Carissa Christensen of BryceTech, named Executive of the Year in the up to $75 million category, and Carey Smith of Parsons, recognized as Executive of the Year in the over $300 million category", "user_id": 1}', 'object': None, 'score': 0.6936542987823486}, {'indexid': 1, 'id': '1', 'text': "2, 2023, — MetroStar's Chief Executive Officer and Co-Founder, Ali Reza Manouchehri, was named a winner of the Greater Washington Government Contractor Awards unveiled on Nov", 'tags': None, 'entry': '2023-11-14 10:49:01.846360', 'data': '{"text": "2, 2023, \\u2014 MetroStar\'s Chief Executive Officer and Co-Founder, Ali Reza Manouchehri

In [ ]:
#TODO tagged inserts loop it in - provide a catch all

In [ ]:

#this wont load on Onyx
#tokenizer = AutoTokenizer.from_pretrained("amazon/MistralLite", model_max_length = 2000)
#model = AutoModelForCausalLM.from_pretrained("amazon/MistralLite", pad_token_id = tokenizer.eos_token_id)

In [ ]:
#TODO simulate chat conversation